[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dhananjayaDev/sadtalker-avatar-generator/blob/main/colab/run_from_github.ipynb)

## Run SadTalker from your GitHub repo

This notebook clones **your** repo [sadtalker-avatar-generator](https://github.com/dhananjayaDev/sadtalker-avatar-generator), installs dependencies, downloads models, and runs the app.

**Before running:** Set **Runtime → Change runtime type → Hardware accelerator → GPU**.

### Step 1: Check GPU

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

### Step 2: Download repo from GitHub

In [ ]:
import os, zipfile, urllib.request
REPO = "dhananjayaDev/sadtalker-avatar-generator"
BRANCH = "main"
zip_url = f"https://github.com/{REPO}/archive/refs/heads/{BRANCH}.zip"
zip_path = "/tmp/repo.zip"
print("Downloading repo...")
urllib.request.urlretrieve(zip_url, zip_path)
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("/content")
os.rename(f"/content/sadtalker-avatar-generator-{BRANCH}", "/content/sadtalker-avatar-generator")
%cd /content/sadtalker-avatar-generator
print("Done. Working directory:", os.getcwd())

### Step 3: Install dependencies

In [ ]:
# Use Colab's pre-installed PyTorch; install only extras (no strict pins = no build errors)
!pip install -q edge-tts face-alignment imageio imageio-ffmpeg librosa resampy pydub kornia yacs scikit-image basicsr facexlib gfpgan av safetensors gradio
!apt-get update -qq && apt-get install -y -qq ffmpeg 2>/dev/null || true

import torch
print(f"PyTorch {torch.__version__}, CUDA: {torch.cuda.is_available()}")

### Step 4: Download checkpoints

In [ ]:
# Run from repo root (we are already in sadtalker-avatar-generator)
!mkdir -p checkpoints
!bash setup/scripts/download_models.sh 2>/dev/null || (echo "Fallback: downloading 256 model only..." && wget -q -nc https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar -O ./checkpoints/mapping_00109-model.pth.tar && wget -q -nc https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar -O ./checkpoints/mapping_00229-model.pth.tar && wget -q -nc https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors -O ./checkpoints/SadTalker_V0.0.2_256.safetensors)
!ls -la checkpoints/

### Step 5: Run the app (Gradio UI)

In [ ]:
# Gradio is already installed in Step 3. Launch app (use repo root so output and link are visible)
%cd /content/sadtalker-avatar-generator
!python sadtalker_local.py --share

---
**Alternative: run inference from command line** (one image + one audio)

Uncomment and run the cell below instead of the Gradio app above.

In [ ]:
# !python inference.py --source_image ./examples/source_image/full_body_1.png --driven_audio ./examples/driven_audio/bus_chinese.wav --result_dir ./results